In [108]:
import numpy as np 
import pandas as pd 
import random 
import os 
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt 
from tqdm import tqdm
import time
import datetime
import pybit
import bybit
import ccxt
import telegram 
import math
import seaborn as sns
import torch 
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

### for tabnet ### 
from pytorch_tabnet.tab_model import TabNetClassifier

### optuna ### 
import optuna
from optuna import Trial, visualization

In [77]:
eth2019 = pd.read_csv("./data_1h/2019_ETHUSDT.csv") 
eth2020 = pd.read_csv("./data_1h/2020_ETHUSDT.csv")  
eth2021 = pd.read_csv("./data_1h/2021_ETHUSDT.csv") 
eth2022 = pd.read_csv("./data_1h/2022_ETHUSDT.csv") 

df = pd.concat([eth2019,eth2020,eth2021,eth2022], axis=0) 
df.index = np.arange(df.shape[0]) 

df = df.drop(columns={'0'})

# Preprocess Data

In [78]:
def create_timestamps(df, ccxt_bybit): 
    dates = df['Open Time'].values 
    ccxt_bybit = ccxt.bybit() 
    timestamp = [] 
    for i in range(len(dates)): 
        date_string = ccxt_bybit.iso8601(int(dates[i]))  
        date_string = date_string[:10] + " " + date_string[11:-5] 
        timestamp.append(date_string)  
    df['timestamp'] = timestamp 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    return df 

In [79]:
def rsi_calc(df, period): 
    df = df['Close'].astype(float) 
    delta = df.diff() 
    gains, declines = delta.copy(), delta.copy() 
    gains[gains < 0] = 0 
    declines[declines > 0] = 0 
    _gains = gains.ewm(com=(period-1), min_periods=period).mean() 
    _loss = declines.abs().ewm(com=(period-1), min_periods=period).mean() 
    RS = _gains / _loss
    return pd.Series((100 - (100 / (1+RS)))/100, name="Scaled_RSI")

In [80]:
def preprocess_df(df, ccxt_bybit): 
    df = create_timestamps(df, ccxt_bybit)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['Hour'] = df['timestamp'].apply(lambda x: x.hour)
    df['Weekday'] = df['timestamp'].apply(lambda x: x.weekday())
    df['Day'] = df['timestamp'].apply(lambda x: x.day)
    df['Month'] = df['timestamp'].apply(lambda x: x.month)
    
    df['RSI'] = rsi_calc(df, period=14)
    
    eps = 1e-10 ### to avoid division by zero for Volume calculations ### 
    
    for window in [5,10,20,30,50,100]: 
        df['close_ma{}'.format(window)] = df['Close'].rolling(window).mean() 
        df['volume_ma{}'.format(window)] = df['Volume'].rolling(window).mean()
        df['close_ma{}_ratio'.format(window)] = (df['Close'] - df['close_ma{}'.format(window)]) / df['close_ma{}'.format(window)] 
        df['volume_ma{}_ratio'.format(window)] = (df['Volume'] - df['volume_ma{}'.format(window)]) / (df['volume_ma{}'.format(window)]+eps)
    
    open_prices = df['Open'].values 
    close_prices = df['Close'].values 
    volumes = df['Volume'].values 
    
    ### strides and differencing ###
    stride_colnames = [] 
    strides = 49 
    for i in range(2, strides): 
        df['close_{}apartclose_ratio'.format(i)] = np.zeros((df.shape[0])) 
        stride_colnames.append('close_{}apartclose_ratio'.format(i)) 
    for i in tqdm(range(strides,df.shape[0]), position=0, leave=True):
        ret = [] 
        for j in range(2,strides):
            ratio = close_prices[i] / close_prices[i-j] 
            df['close_{}apartclose_ratio'.format(j)].iloc[i] = ratio 
        
    labels = [] 
    for i in range(len(close_prices)-1): 
        ret = close_prices[i+1] / close_prices[i] 
        if ret > 1.0:  
            labels.append(1) 
        elif ret <= 1.0: 
            labels.append(0) 
    labels.append(None)
    df['Labels'] = labels 
    
    df['high_close_ratio'] = (df['High'].values - df['Close'].values) / df['Close'].values 
    df['low_close_ratio'] = (df['Low'].values - df['Close'].values) / df['Close'].values  
    
    close_lastclose_ratio = [None] 
    for i in range(1, len(close_prices)): 
        ratio = close_prices[i] / close_prices[i-1] 
        close_lastclose_ratio.append(ratio) 
    df['close_lastclose_ratio'] = close_lastclose_ratio 
    
    volume_lastvolume_ratio = [None] 
    for i in range(1, len(volumes)):
        ratio = volumes[i] / (volumes[i-1]+eps) 
        volume_lastvolume_ratio.append(ratio) 
    df['volume_lastvolume_ratio'] = volume_lastvolume_ratio
    
    cols = ["Hour",
            "Day", 
            "Month",
            "Weekday", 
            "close_ma5_ratio", 
            "volume_ma5_ratio",
            "close_ma10_ratio", 
            "volume_ma10_ratio",
            "close_ma20_ratio", 
            "volume_ma20_ratio", 
            "close_ma30_ratio", 
            "volume_ma30_ratio", 
            "close_ma50_ratio", 
            "volume_ma50_ratio",
            "close_ma100_ratio",
            "volume_ma100_ratio", 
            "high_close_ratio",
            "low_close_ratio", 
            "close_lastclose_ratio", 
            "volume_lastvolume_ratio", 
            "Labels"] 
    
    cols = cols + stride_colnames 
    df = df[cols] # specify feature columns  
    return df

ccxt_bybit = ccxt.bybit() 

df = preprocess_df(df, ccxt_bybit) 

df = df.dropna() 

df.to_csv("feature_engineered_ethusdt.csv",index=False)

df.head()

  0%|          | 0/26932 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
100%|██████████| 26932/26932 [04:04<00:00, 110.14it/s]


,Hour,Day,Month,Weekday,close_ma5_ratio,volume_ma5_ratio,close_ma10_ratio,volume_ma10_ratio,close_ma20_ratio,volume_ma20_ratio,...,close_39apartclose_ratio,close_40apartclose_ratio,close_41apartclose_ratio,close_42apartclose_ratio,close_43apartclose_ratio,close_44apartclose_ratio,close_45apartclose_ratio,close_46apartclose_ratio,close_47apartclose_ratio,close_48apartclose_ratio
99,2,5,1,5,0.013170,-0.311798,0.033473,-0.090164,0.042926,-0.044944,...,1.074403,1.061505,1.063585,1.059861,1.036891,1.036003,1.037437,1.034030,1.037027,1.042177
100,3,5,1,5,0.006966,-0.523102,0.027567,-0.400415,0.042274,-0.352542,...,1.062226,1.075495,1.062584,1.064667,1.060939,1.037945,1.037057,1.038492,1.035081,1.038081
101,4,5,1,5,-0.003023,-0.478648,0.017314,-0.371902,0.036570,-0.274155,...,1.058736,1.058451,1.071672,1.058808,1.060882,1.057168,1.034256,1.033371,1.034801,1.031402
102,4,5,1,5,-0.002034,-0.286171,0.011617,-0.385466,0.034279,-0.273873,...,1.069118,1.058736,1.058451,1.071672,1.058808,1.060882,1.057168,1.034256,1.033371,1.034801
103,5,5,1,5,0.002237,-0.170133,0.011619,-0.470016,0.036758,-0.368733,...,1.067280,1.073749,1.063322,1.063035,1.076314,1.063394,1.065477,1.061747,1.038735,1.037846


In [103]:
train_cols = ['Hour', 'Day', 'Month', 'Weekday', 'close_ma5_ratio',
       'volume_ma5_ratio', 'close_ma10_ratio', 'volume_ma10_ratio',
       'close_ma20_ratio', 'volume_ma20_ratio', 'close_ma30_ratio',
       'volume_ma30_ratio', 'close_ma50_ratio', 'volume_ma50_ratio',
       'close_ma100_ratio', 'volume_ma100_ratio', 'high_close_ratio',
       'low_close_ratio', 'close_lastclose_ratio', 'volume_lastvolume_ratio',
       'close_2apartclose_ratio', 'close_3apartclose_ratio',
       'close_4apartclose_ratio', 'close_5apartclose_ratio',
       'close_6apartclose_ratio', 'close_7apartclose_ratio',
       'close_8apartclose_ratio', 'close_9apartclose_ratio',
       'close_10apartclose_ratio', 'close_11apartclose_ratio',
       'close_12apartclose_ratio', 'close_13apartclose_ratio',
       'close_14apartclose_ratio', 'close_15apartclose_ratio',
       'close_16apartclose_ratio', 'close_17apartclose_ratio',
       'close_18apartclose_ratio', 'close_19apartclose_ratio',
       'close_20apartclose_ratio', 'close_21apartclose_ratio',
       'close_22apartclose_ratio', 'close_23apartclose_ratio',
       'close_24apartclose_ratio', 'close_25apartclose_ratio',
       'close_26apartclose_ratio', 'close_27apartclose_ratio',
       'close_28apartclose_ratio', 'close_29apartclose_ratio',
       'close_30apartclose_ratio', 'close_31apartclose_ratio',
       'close_32apartclose_ratio', 'close_33apartclose_ratio',
       'close_34apartclose_ratio', 'close_35apartclose_ratio',
       'close_36apartclose_ratio', 'close_37apartclose_ratio',
       'close_38apartclose_ratio', 'close_39apartclose_ratio',
       'close_40apartclose_ratio', 'close_41apartclose_ratio',
       'close_42apartclose_ratio', 'close_43apartclose_ratio',
       'close_44apartclose_ratio', 'close_45apartclose_ratio',
       'close_46apartclose_ratio', 'close_47apartclose_ratio',
       'close_48apartclose_ratio']

target_cols = ['Labels'] 

X = df[train_cols] 
Y = df[target_cols] 

X.shape, Y.shape, df.shape

((26881, 67), (26881, 1), (26881, 68))

In [117]:
X = X.values 
Y = Y.values 

In [120]:
Y = Y.reshape((-1)) 

X.shape, Y.shape

((26881, 67), (26881,))

# Tune TabNet with Optuna

In [123]:
def Objective(trial): 
    mask_type = trial.suggest_categorical('mask_type', ["entmax", "sparsemax"]) 
    n_da = trial.suggest_int("n_da", 8, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 3, 10, step=1) 
    gamma = trial.suggest_float("gamma", 1.0, 2.0, step = 0.2) 
    n_shared = trial.suggest_int("n_shared", 1, 5) 
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True) 
    
    tabnet_params = dict(n_d = n_da, 
                         n_a = n_da, 
                         n_steps = n_steps, 
                         gamma = gamma, 
                         lambda_sparse = lambda_sparse, 
                         optimizer_fn = torch.optim.Adam,
                         optimizer_params = dict(lr=2e-2, weight_decay=1e-5), 
                         mask_type = mask_type,
                         n_shared = n_shared, 
                         scheduler_params = dict(mode="min", 
                                                 patience = trial.suggest_int("patienceScheduler", low = 3, high = 10),
                                                 min_lr = 1e-5,
                                                 factor = 0.5),
                         scheduler_fn = torch.optim.lr_scheduler.ReduceLROnPlateau, 
                         verbose = 1) 
    
    
    kf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    cv_score_array = [] 
    for (train_idx, val_idx) in kf.split(X): 
        X_train, X_val = X[train_idx], X[val_idx] 
        Y_train, Y_val = Y[train_idx], Y[val_idx] 
        clf = TabNetClassifier(**tabnet_params) 
        clf.fit(X_train = X_train, 
                y_train = Y_train, 
                eval_set = [(X_val, Y_val)], 
                patience = trial.suggest_int("patience", low=20, high=60), max_epochs = trial.suggest_int('epochs',100,1000), 
                eval_metric = ["auc", "accuracy"], 
                drop_last = True) 
        cv_score_array.append(clf.best_cost) 
    avg = np.mean(cv_score_array) 
    return avg 

In [124]:
study = optuna.create_study(direction = "maximize", study_name = "TabNet_optimization_clf") 

study.optimize(Objective, timeout = 6*60)

[I 2022-01-10 12:50:56,102] A new study created in memory with name: TabNet_optimization_clf


Device used : cuda
epoch 0  | loss: 1.51812 | val_0_auc: 0.49964 | val_0_accuracy: 0.49805 |  0:00:03s
epoch 1  | loss: 0.94099 | val_0_auc: 0.50054 | val_0_accuracy: 0.49135 |  0:00:06s
epoch 2  | loss: 0.8692  | val_0_auc: 0.49531 | val_0_accuracy: 0.50046 |  0:00:09s
epoch 3  | loss: 0.75618 | val_0_auc: 0.50463 | val_0_accuracy: 0.49284 |  0:00:12s
epoch 4  | loss: 0.71537 | val_0_auc: 0.50116 | val_0_accuracy: 0.5079  |  0:00:15s
epoch 5  | loss: 0.69864 | val_0_auc: 0.49672 | val_0_accuracy: 0.50642 |  0:00:18s
epoch 6  | loss: 0.6944  | val_0_auc: 0.50603 | val_0_accuracy: 0.50772 |  0:00:22s
epoch 7  | loss: 0.69478 | val_0_auc: 0.50438 | val_0_accuracy: 0.49954 |  0:00:25s
epoch 8  | loss: 0.69292 | val_0_auc: 0.50571 | val_0_accuracy: 0.50939 |  0:00:28s
epoch 9  | loss: 0.69236 | val_0_auc: 0.49915 | val_0_accuracy: 0.50921 |  0:00:31s
epoch 10 | loss: 0.69225 | val_0_auc: 0.51156 | val_0_accuracy: 0.50976 |  0:00:34s
epoch 11 | loss: 0.69125 | val_0_auc: 0.50746 | val_0_acc

epoch 98 | loss: 0.68647 | val_0_auc: 0.53986 | val_0_accuracy: 0.53338 |  0:05:00s
epoch 99 | loss: 0.68582 | val_0_auc: 0.53958 | val_0_accuracy: 0.53152 |  0:05:03s
epoch 100| loss: 0.68625 | val_0_auc: 0.5396  | val_0_accuracy: 0.53394 |  0:05:06s
epoch 101| loss: 0.68648 | val_0_auc: 0.53942 | val_0_accuracy: 0.52948 |  0:05:09s
epoch 102| loss: 0.68596 | val_0_auc: 0.54021 | val_0_accuracy: 0.5319  |  0:05:12s
epoch 103| loss: 0.68629 | val_0_auc: 0.53874 | val_0_accuracy: 0.53487 |  0:05:15s
epoch 104| loss: 0.68605 | val_0_auc: 0.53913 | val_0_accuracy: 0.53487 |  0:05:18s
epoch 105| loss: 0.68554 | val_0_auc: 0.53887 | val_0_accuracy: 0.53208 |  0:05:20s
epoch 106| loss: 0.68614 | val_0_auc: 0.53883 | val_0_accuracy: 0.53115 |  0:05:23s
epoch 107| loss: 0.68572 | val_0_auc: 0.53951 | val_0_accuracy: 0.5319  |  0:05:26s
epoch 108| loss: 0.6858  | val_0_auc: 0.5384  | val_0_accuracy: 0.53115 |  0:05:29s
epoch 109| loss: 0.68661 | val_0_auc: 0.53958 | val_0_accuracy: 0.53208 |  0

epoch 13 | loss: 0.69268 | val_0_auc: 0.51691 | val_0_accuracy: 0.51451 |  0:00:42s
epoch 14 | loss: 0.69184 | val_0_auc: 0.51056 | val_0_accuracy: 0.51656 |  0:00:45s
epoch 15 | loss: 0.6932  | val_0_auc: 0.51573 | val_0_accuracy: 0.51711 |  0:00:48s
epoch 16 | loss: 0.69265 | val_0_auc: 0.51626 | val_0_accuracy: 0.52176 |  0:00:51s
epoch 17 | loss: 0.69175 | val_0_auc: 0.5281  | val_0_accuracy: 0.52288 |  0:00:54s
epoch 18 | loss: 0.69151 | val_0_auc: 0.52557 | val_0_accuracy: 0.51972 |  0:00:57s
epoch 19 | loss: 0.69071 | val_0_auc: 0.52967 | val_0_accuracy: 0.52418 |  0:01:00s
epoch 20 | loss: 0.69161 | val_0_auc: 0.52613 | val_0_accuracy: 0.52586 |  0:01:03s
epoch 21 | loss: 0.6912  | val_0_auc: 0.52346 | val_0_accuracy: 0.52455 |  0:01:06s
epoch 22 | loss: 0.69118 | val_0_auc: 0.5244  | val_0_accuracy: 0.52511 |  0:01:09s
epoch 23 | loss: 0.69023 | val_0_auc: 0.52534 | val_0_accuracy: 0.52511 |  0:01:12s
epoch 24 | loss: 0.6899  | val_0_auc: 0.53299 | val_0_accuracy: 0.52641 |  0

epoch 24 | loss: 0.6879  | val_0_auc: 0.5293  | val_0_accuracy: 0.51283 |  0:01:15s
epoch 25 | loss: 0.68837 | val_0_auc: 0.53198 | val_0_accuracy: 0.50986 |  0:01:18s
epoch 26 | loss: 0.68905 | val_0_auc: 0.52848 | val_0_accuracy: 0.51767 |  0:01:22s
epoch 27 | loss: 0.688   | val_0_auc: 0.5284  | val_0_accuracy: 0.51302 |  0:01:25s
epoch 28 | loss: 0.68816 | val_0_auc: 0.52499 | val_0_accuracy: 0.51581 |  0:01:28s
epoch 29 | loss: 0.688   | val_0_auc: 0.5306  | val_0_accuracy: 0.51656 |  0:01:31s
epoch 30 | loss: 0.68745 | val_0_auc: 0.52786 | val_0_accuracy: 0.51581 |  0:01:35s
epoch 31 | loss: 0.68765 | val_0_auc: 0.53293 | val_0_accuracy: 0.51953 |  0:01:38s
epoch 32 | loss: 0.68724 | val_0_auc: 0.53391 | val_0_accuracy: 0.52158 |  0:01:41s
epoch 33 | loss: 0.68775 | val_0_auc: 0.53694 | val_0_accuracy: 0.52548 |  0:01:45s
epoch 34 | loss: 0.68726 | val_0_auc: 0.53652 | val_0_accuracy: 0.52548 |  0:01:48s
epoch 35 | loss: 0.68725 | val_0_auc: 0.5362  | val_0_accuracy: 0.5279  |  0

epoch 47 | loss: 0.68713 | val_0_auc: 0.54694 | val_0_accuracy: 0.52995 |  0:02:36s
epoch 48 | loss: 0.68712 | val_0_auc: 0.54595 | val_0_accuracy: 0.52995 |  0:02:39s
epoch 49 | loss: 0.68737 | val_0_auc: 0.5461  | val_0_accuracy: 0.53367 |  0:02:42s
epoch 50 | loss: 0.68688 | val_0_auc: 0.54702 | val_0_accuracy: 0.53274 |  0:02:45s
epoch 51 | loss: 0.68671 | val_0_auc: 0.54558 | val_0_accuracy: 0.53292 |  0:02:48s
epoch 52 | loss: 0.6873  | val_0_auc: 0.54719 | val_0_accuracy: 0.5346  |  0:02:51s
epoch 53 | loss: 0.68724 | val_0_auc: 0.54635 | val_0_accuracy: 0.53255 |  0:02:54s
epoch 54 | loss: 0.6872  | val_0_auc: 0.54476 | val_0_accuracy: 0.53311 |  0:02:57s
epoch 55 | loss: 0.68677 | val_0_auc: 0.54586 | val_0_accuracy: 0.53088 |  0:03:00s
epoch 56 | loss: 0.68688 | val_0_auc: 0.54564 | val_0_accuracy: 0.53274 |  0:03:03s
epoch 57 | loss: 0.68619 | val_0_auc: 0.54674 | val_0_accuracy: 0.53274 |  0:03:06s
epoch 58 | loss: 0.68725 | val_0_auc: 0.5464  | val_0_accuracy: 0.53144 |  0

[I 2022-01-10 13:14:59,627] Trial 0 finished with value: 0.5412374935793547 and parameters: {'mask_type': 'entmax', 'n_da': 56, 'n_steps': 9, 'gamma': 1.8, 'n_shared': 1, 'lambda_sparse': 0.0006887580992842807, 'patienceScheduler': 5, 'patience': 30, 'epochs': 563}. Best is trial 0 with value: 0.5412374935793547.


In [125]:
TabNet_params = study.best_params 
TabNet_params 

{'mask_type': 'entmax',
 'n_da': 56,
 'n_steps': 9,
 'gamma': 1.8,
 'n_shared': 1,
 'lambda_sparse': 0.0006887580992842807,
 'patienceScheduler': 5,
 'patience': 30,
 'epochs': 563}

In [126]:
final_params = dict(n_d = TabNet_params['n_da'], 
                    n_a = TabNet_params['n_da'], 
                    n_steps = TabNet_params['n_steps'],
                    gamma = TabNet_params['gamma'],
                    lambda_sparse = TabNet_params['lambda_sparse'],
                    optimizer_fn = torch.optim.Adam,
                    optimizer_params = dict(lr=2e-2, 
                                            weight_decay=1e-5),
                    mask_type = TabNet_params['mask_type'], 
                    n_shared = TabNet_params['n_shared'],
                    scheduler_params = dict(mode = "min",
                                            patience = TabNet_params['patienceScheduler'],
                                            min_lr = 1e-5,
                                            factor = 0.5),
                    scheduler_fn = torch.optim.lr_scheduler.ReduceLROnPlateau,
                    verbose = 1) 

epochs = TabNet_params['epochs']

In [128]:
best_clf = TabNetClassifier(**final_params) 

best_clf.fit(X_train = X, 
             y_train = Y, 
             patience = TabNet_params['patience'], 
             max_epochs = epochs, 
             eval_metric = ['auc', 'accuracy'])  

Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.24382 |  0:00:03s
epoch 1  | loss: 1.13815 |  0:00:06s
epoch 2  | loss: 0.76559 |  0:00:10s
epoch 3  | loss: 0.69471 |  0:00:13s
epoch 4  | loss: 0.6948  |  0:00:17s
epoch 5  | loss: 0.69536 |  0:00:20s
epoch 6  | loss: 0.69639 |  0:00:23s
epoch 7  | loss: 0.69331 |  0:00:27s
epoch 8  | loss: 0.69428 |  0:00:30s
epoch 9  | loss: 0.69406 |  0:00:34s
epoch 10 | loss: 0.695   |  0:00:37s
epoch 11 | loss: 0.6928  |  0:00:40s
epoch 12 | loss: 0.69219 |  0:00:44s
epoch 13 | loss: 0.69208 |  0:00:47s
epoch 14 | loss: 0.69281 |  0:00:51s
epoch 15 | loss: 0.69308 |  0:00:54s
epoch 16 | loss: 0.69185 |  0:00:58s
epoch 17 | loss: 0.6934  |  0:01:01s
epoch 18 | loss: 0.69189 |  0:01:05s
epoch 19 | loss: 0.6922  |  0:01:08s
epoch 20 | loss: 0.69133 |  0:01:12s
epoch 21 | loss: 0.69222 |  0:01:16s
epoch 22 | loss: 0.69187 |  0:01:19s
epoch 23 | loss: 0.6918  |  0:01:23s
epoch 24 | loss: 0.6

epoch 219| loss: 0.68751 |  0:12:44s
epoch 220| loss: 0.68988 |  0:12:47s
epoch 221| loss: 0.68811 |  0:12:50s
epoch 222| loss: 0.68704 |  0:12:54s
epoch 223| loss: 0.68688 |  0:12:57s
epoch 224| loss: 0.68654 |  0:13:01s
epoch 225| loss: 0.68597 |  0:13:04s
epoch 226| loss: 0.68559 |  0:13:07s
epoch 227| loss: 0.68471 |  0:13:11s
epoch 228| loss: 0.68529 |  0:13:15s
epoch 229| loss: 0.68553 |  0:13:19s
epoch 230| loss: 0.68463 |  0:13:23s
epoch 231| loss: 0.68373 |  0:13:27s
epoch 232| loss: 0.68681 |  0:13:31s
epoch 233| loss: 0.68524 |  0:13:35s
epoch 234| loss: 0.68449 |  0:13:39s
epoch 235| loss: 0.68412 |  0:13:43s
epoch 236| loss: 0.68433 |  0:13:47s
epoch 237| loss: 0.68467 |  0:13:52s
epoch 238| loss: 0.68336 |  0:13:56s
epoch 239| loss: 0.68365 |  0:14:00s
epoch 240| loss: 0.68417 |  0:14:04s
epoch 241| loss: 0.68375 |  0:14:07s
epoch 242| loss: 0.68405 |  0:14:11s
epoch 243| loss: 0.6838  |  0:14:15s
epoch 244| loss: 0.68312 |  0:14:18s
epoch 245| loss: 0.68367 |  0:14:22s
e

epoch 441| loss: 0.6583  |  0:25:53s
epoch 442| loss: 0.66028 |  0:25:57s
epoch 443| loss: 0.66111 |  0:26:01s
epoch 444| loss: 0.65935 |  0:26:05s
epoch 445| loss: 0.65782 |  0:26:10s
epoch 446| loss: 0.65852 |  0:26:14s
epoch 447| loss: 0.65641 |  0:26:18s
epoch 448| loss: 0.65833 |  0:26:21s
epoch 449| loss: 0.65649 |  0:26:24s
epoch 450| loss: 0.65778 |  0:26:28s
epoch 451| loss: 0.65764 |  0:26:31s
epoch 452| loss: 0.65701 |  0:26:34s
epoch 453| loss: 0.65737 |  0:26:38s
epoch 454| loss: 0.65806 |  0:26:42s
epoch 455| loss: 0.65464 |  0:26:46s
epoch 456| loss: 0.65805 |  0:26:50s
epoch 457| loss: 0.65592 |  0:26:53s
epoch 458| loss: 0.6564  |  0:26:57s
epoch 459| loss: 0.65919 |  0:27:00s
epoch 460| loss: 0.65602 |  0:27:04s
epoch 461| loss: 0.65409 |  0:27:07s
epoch 462| loss: 0.65427 |  0:27:11s
epoch 463| loss: 0.65337 |  0:27:14s
epoch 464| loss: 0.65777 |  0:27:18s
epoch 465| loss: 0.65854 |  0:27:21s
epoch 466| loss: 0.65358 |  0:27:25s
epoch 467| loss: 0.65792 |  0:27:29s
e

In [129]:
### save model ### 
best_clf.save_model("tabnet_ethusdt")

Successfully saved model at tabnet_ethusdt.zip


'tabnet_ethusdt.zip'

# Predict on holdout set